# Fig6

In [1]:
from zebrafish_ms2_paper.simulation import Params, simulate, sim_ms2, delay_protein
from zebrafish_ms2_paper.trace_analysis import binarize_trace, get_on_and_off_times, get_burst_durations, get_burst_inactive_durations, extract_traces
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import numpy as np
from multiprocessing import Pool
import pandas as pd
import pickle


## plot style

In [170]:
%matplotlib qt

In [2]:
def color_palette():
    """
    Returns a dictionary of the PBOC color palette
    """
    return {'green': '#7AA974', 'light_green': '#BFD598',
              'pale_green': '#DCECCB', 'yellow': '#EAC264',
              'light_yellow': '#F3DAA9', 'pale_yellow': '#FFEDCE',
              'blue': '#738FC1', 'light_blue': '#A9BFE3',
              'pale_blue': '#C9D7EE', 'red': '#D56C55', 'light_red': '#E8B19D',
              'pale_red': '#F1D4C9', 'purple': '#AB85AC',
              'light_purple': '#D4C2D9', 'dark_green':'#7E9D90', 'dark_brown':'#905426'}


def style_axes(ax, fontsize=24, labelsize=20):
    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='x', color='white', length=10, width=2)
    ax.tick_params(axis='y', color='white', length=10, width=2)

    return ax

    
pboc_rc = {'axes.facecolor': '#E3DCD0',
          'font.family': 'Lucida Sans',
          'grid.linestyle': '-',
          'grid.linewidth': 0.5,
          'grid.alpha': 0.75,
          'grid.color': '#ffffff',
          'ytick.direction': 'in',
          'xtick.direction': 'in',
          'ytick.major.width':5,
          'xtick.major.width':5,
          'ytick.major.size': 5,
          'xtick.major.size': 5,
          'mathtext.fontset': 'stixsans',
          'mathtext.sf': 'sans',
          'legend.frameon': True,
          'legend.facecolor': '#FFEDCE',
          'figure.dpi': 150,
           'xtick.color': 'k',
           'ytick.color': 'k'}


rcParams.update(pboc_rc)
colors = color_palette()

linewidth = 4
fontweight = 'bold'
fontsize = 24
rc('axes', linewidth=4)


## load the data

In [110]:
"""specify the paths to the required datasets"""
path_to_dataset1 = r'/media/brandon/Data1/Somitogenesis/Dorado/gauss_001_v2_sigma_filt/filtered_df.pkl'

path_to_non_blank_timepoints = r'/media/brandon/Data1/Somitogenesis/Dorado/non_blank_timepoints_full.pkl'

## Fig6B,C,D
Plot example traces for the three burst regulation models. Here we also plot the predicted protein traces, though we removed these from the final figure for simplicity. This code generates new stochastic simulations, so the traces won't be exactly the same was what is plotted in the figure.

In [3]:
"""fixed model params"""
elongation_time = 0.29
delta_t = 1.0
w = elongation_time / delta_t
ms2_conversion_value = 62363
conversion_factor = 2 * ms2_conversion_value / 40 / 0.23 / elongation_time
window_size = 3
sigma = 0.2
Tmax = 10_000
bins = np.arange(window_size, 61)
n_replicates = 20
y_max = 0.15
translation_rate = 4.5
hill_coefficient = 12
mrna_decay_rate = 0.23
protein_decay_rate = 0.23
dt = 0.01
max_time_to_next_reaction = 1.0
n_bootstraps = 100


In [153]:
def run_example_simulation_and_plot(p, burn_in_time=0, state_linewidth=4):
    state, mrna, protein, tvec, p = simulate(p)
    p_tau = (p.transcription_rate_1 > 0) * delay_protein(protein, tvec, p.delay)
    loading_rate = p.transcription_rate_0 + p.transcription_rate_1 * 1 / (1 + (p_tau / p.KD) ** p.n)
    detection_threshold = 10_000#0
    ms2, t_obs = sim_ms2(state, tvec, loading_rate, w, delta_t, sigma, detection_threshold, conversion_factor)
    
    start_id = np.where(tvec > burn_in_time)[0][0]
    state = state[start_id:]
    mrna = mrna[start_id:]
    protein = protein[start_id:]
    tvec = tvec[start_id:] - burn_in_time

    start_id = np.where(t_obs > burn_in_time)[0][0]
    ms2 = ms2[start_id:]
    t_obs = t_obs[start_id:]
    
    """plot"""
    plt.figure(figsize=(7, 10))

    # protein
    plt.subplot(3,1,1)
    #plt.plot(tvec, protein / np.max(protein), '-', linewidth=4, color=colors['red'])
    plt.plot(tvec, protein, '-', linewidth=4, color=colors['red'])
  
    plt.ylabel('protein (a.u.)', fontsize=fontsize)
    plt.xlabel('time (min)', fontsize=fontsize)
    #plt.ylim([0,1.2])
    plt.xticks([0, 60, 120, 180])
    ax = plt.gca()
    ax = style_axes(ax)
    
    # state
    plt.subplot(3,1,2)
    plt.plot(tvec, state, '-', linewidth=state_linewidth, color='k')
    
    plt.ylabel('promoter state', fontsize=fontsize)
    plt.xlabel('time (min)', fontsize=fontsize)
    plt.ylim([0,1])
    plt.xticks([0, 60, 120, 180])
    plt.yticks([0, 1], labels=['OFF', 'ON'])
    ax = plt.gca()
    ax = style_axes(ax)
    
    plt.tight_layout()
    
    # MS2
    plt.subplot(3,1,3)
    #plt.plot(t_obs, ms2 / np.max(ms2), '-', linewidth=4, color=colors['green'])
    plt.plot(t_obs, ms2, '-', linewidth=4, color=colors['green'])
    state = binarize_trace(ms2, t_obs, thresh=1.0, window_size=window_size)
    plt.plot(t_obs, state * np.max(ms2), 'k-')
    
    plt.ylabel('predicted MS2 \nfluorescence (a.u.)', fontsize=fontsize)
    plt.xlabel('time (min)', fontsize=fontsize)
    #plt.ylim([0,1.2])
    plt.xticks([0, 60, 120, 180])
    ax = plt.gca()
    ax = style_axes(ax)
    
    plt.tight_layout()
    
    
    
    plt.tight_layout()

In [180]:
"""amplitude regulation"""
p = Params()

# fixed params
p.translation_rate = translation_rate
p.mrna_decay_rate = mrna_decay_rate
p.protein_decay_rate = protein_decay_rate
p.Tmax = 200
p.dt = dt
p.max_time_to_next_reaction = max_time_to_next_reaction
p.n = 5#hill_coefficient

# model specific params
p.transcription_rate_0 = 0.0
p.transcription_rate_1 = 20.0
p.k_on0 = 0.04
p.k_off0 = 0.08
p.k_on1 = 0.0
p.k_off1 = 0.0
p.KD = 500.0
p.delay = 8




run_example_simulation_and_plot(p, state_linewidth=1.0)

In [181]:
plt.savefig(r'/home/brandon/Documents/somitogenesis/paper_1/draft_1/figures/v2/theory_fig/simulated_traces_v2/amp_reg_example_3panel_v3.pdf')

In [139]:
"""k_on regulation"""
p = Params()

# fixed params
p.translation_rate = translation_rate
p.mrna_decay_rate = mrna_decay_rate
p.protein_decay_rate = protein_decay_rate
p.Tmax = 240
p.dt = dt
p.max_time_to_next_reaction = max_time_to_next_reaction
p.n = hill_coefficient

# model specific params
p.transcription_rate_0 = 10.0
p.transcription_rate_1 = 0.0
p.KD = 1
p.n = 12
p.k_off0 = 2 * 0.08
p.k_on1 = 100 * 0.03 / p.KD ** p.n
p.k_off1 = 0.0
p.delay = 0

run_example_simulation_and_plot(p)

In [140]:
plt.savefig(r'/home/brandon/Documents/somitogenesis/paper_1/draft_1/figures/v2/theory_fig/simulated_traces_v2/kon_reg_example_3panel_v2.pdf')

In [142]:
"""k_off regulation"""
p = Params()

# fixed params
p.translation_rate = translation_rate
p.mrna_decay_rate = mrna_decay_rate
p.protein_decay_rate = protein_decay_rate
p.Tmax = 240
p.dt = dt
p.max_time_to_next_reaction = max_time_to_next_reaction
p.n = hill_coefficient

# model specific params
p.transcription_rate_0 = 10.0
p.transcription_rate_1 = 0.0
p.k_on0 = 0.08
p.k_off0 = 0.0
p.k_on1 = 0.0
p.k_off1 = 0.2
p.KD = 500.0
p.delay = 0

run_example_simulation_and_plot(p)


In [143]:
plt.savefig(r'/home/brandon/Documents/somitogenesis/paper_1/draft_1/figures/v2/theory_fig/simulated_traces_v2/koff_reg_example_3panel_v3.pdf')

## Fig6E
For the three models of burst auto-regulation and one set of parameters each, compute the predicted interval distributions. We use multiprocessing for speed, but this can be circumvented by simply calling func below with a longer Tmax. 

In [112]:
%matplotlib qt

In [154]:
"""function for computing interval distributions"""
def func(p):
    state, mrna, protein, tvec, p = simulate(p)
    burn_in_time = 45
    start_id = np.where(tvec > burn_in_time)[0][0]
    state = state[start_id:]
    mrna = mrna[start_id:]
    protein = protein[start_id:]
    tvec = tvec[start_id:] - burn_in_time

    p_tau = (p.transcription_rate_1 > 0) * delay_protein(protein, tvec, p.delay)
    production_rate = (p.transcription_rate_0 + p.transcription_rate_1) * 1 / (1 + (p_tau / p.KD) ** p.n)

    detection_threshold = 10_000#0
    ms2, t_obs = sim_ms2(state, tvec, production_rate, w, delta_t, sigma, detection_threshold, conversion_factor)

    start_id = np.where(t_obs > burn_in_time)[0][0]
    ms2 = ms2[start_id:]
    t_obs = t_obs[start_id:] 

    # use a detection threshold to produce experimental fraction of time active
    # detection_threshold = fit_detection_threshold(ms2, t_obs, start_quant=0.5, target_binarized_quant=0.486, quant_step=0.001)
    # ms2[ms2 < detection_threshold] = 0

    state = binarize_trace(ms2, t_obs, thresh=1.0, window_size=window_size)
    on_times, off_times = get_on_and_off_times(state, t_obs)
    active_durations  = get_burst_durations(on_times, off_times)
    inactive_durations = get_burst_inactive_durations(on_times, off_times)
    
    return active_durations, inactive_durations

def init_pool_processes():
    np.random.seed()
    

def fit_detection_threshold(ms2, t_obs, start_quant, target_binarized_quant, quant_step):
    ms2_thresh = ms2.copy()
    quant = start_quant
    ms2, thresh, fraction_of_time_transcribing, target_met = fit_detection_threshold_update(ms2_thresh, t_obs, quant, target_binarized_quant)
    
    while not target_met:
        quant += quant_step
        ms2, thresh, fraction_of_time_transcribing, target_met = fit_detection_threshold_update(ms2_thresh, t_obs, quant, target_binarized_quant)
        
    return thresh


def fit_detection_threshold_update(ms2, t_obs, quant, target_binarized_quant):
    thresh = np.quantile(ms2, q=quant)
    ms2[ms2 < thresh] = 0
    state = binarize_trace(ms2, t_obs, thresh=1.0, window_size=window_size)
    fraction_of_time_transcribing = np.sum(state) / len(state)
    target_met = fraction_of_time_transcribing < target_binarized_quant
    
    return ms2, thresh, fraction_of_time_transcribing, target_met



In [155]:
"""amplitude regulation"""
p = Params()
save = True

# fixed params
p.translation_rate = translation_rate
p.mrna_decay_rate = mrna_decay_rate
p.protein_decay_rate = protein_decay_rate
p.Tmax = 500
p.dt = dt
p.max_time_to_next_reaction = max_time_to_next_reaction
p.n = 5#hill_coefficient

# model specific params
p.transcription_rate_0 = 0.0
p.transcription_rate_1 = 20.0
p.k_on0 = 0.04
p.k_off0 = 0.08
p.k_on1 = 0.0
p.k_off1 = 0.0
p.KD = 500.0
p.delay = 8


p_arr = [p] * n_replicates * 20

with Pool(processes=15, initializer=init_pool_processes) as pool:
    res = pool.map(func, p_arr)

active_durations = [item for sublist in res for item in sublist[0]]
inactive_durations = [item for sublist in res for item in sublist[1]]

"""save the simulation output"""
if save:
    output = [active_durations, inactive_durations]
    output_file = r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/amp_reg_v2.pkl'
    with open(output_file, 'wb') as f:
        pickle.dump(output, f)
    





In [182]:
"""plot amp reg"""
load = True
# load data
if load:
    output_file = r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/amp_reg_v2.pkl'
    with open(output_file, 'rb') as f:
        active_durations, inactive_durations = pickle.load(f)

sim_active_dist_arr = bootstrap_simulated_distributions(active_durations, bins, n_bootstraps)
sim_inactive_dist_arr = bootstrap_simulated_distributions(inactive_durations, bins, n_bootstraps)

y_max = 0.2#0.32   
yticks = [0, 0.10, 0.20]#, 0.30]
plt.figure(figsize=(10, 5.5))
# plot active durations
plt.subplot(121)
counts, bins = np.histogram(active_durations, bins=bins)
prob_dens = counts / np.sum(counts) / np.diff(bins)
s = np.std(sim_active_dist_arr, axis=0)
plt.plot(bins[:-1], prob_dens, '-', linewidth=3, color=colors['green'])
plt.plot(bins[:-1], prob_dens, 'ko', markersize=16, markerfacecolor=colors['green'], alpha=0.75, label='data', linewidth=2)
plt.errorbar(bins[:-1], prob_dens, s, color='none',ecolor='k', linewidth=2, barsabove=True, capsize=4)

k = 1 / np.mean(active_durations)
x = np.linspace(0, 200, 1000)
y = k * np.exp(-k * x)
# plt.plot(x, y, 'k-', linewidth=4, label='stochastic bursts')

plt.ylim([0, y_max])
plt.xlim([-5, 65])
plt.xticks([0, 30, 60])
plt.yticks(yticks)
plt.xlabel('active interval (min)', fontsize=fontsize)
plt.ylabel('\n probability density (min)$^{-1}$', fontsize=fontsize)
ax = plt.gca()
ax = style_axes(ax)

      
#plt.tight_layout()

plt.subplot(122)
counts, bins = np.histogram(inactive_durations, bins=bins)
prob_dens = counts / np.sum(counts) / np.diff(bins)
s = np.std(sim_inactive_dist_arr, axis=0)
plt.plot(bins[:-1], prob_dens, '-', linewidth=3, color=colors['green'])
plt.plot(bins[:-1], prob_dens, 'ko', markersize=16, markerfacecolor=colors['green'], alpha=0.75, label='data', linewidth=2)
plt.errorbar(bins[:-1], prob_dens, s, color='none',ecolor='k', linewidth=2, barsabove=True, capsize=4)

k = 1 / np.mean(inactive_durations)
x = np.linspace(0, 200, 1000)
y = k * np.exp(-k * x)
# plt.plot(x, y, 'k-', linewidth=4, label='stochastic bursts')

plt.ylim([0, y_max])
plt.xlim([-5, 65])
plt.xticks([0, 30, 60])
plt.yticks(yticks)
plt.xlabel('inactive interval (min)', fontsize=fontsize)
plt.ylabel('\n probability density (min)$^{-1}$', fontsize=fontsize)
ax = plt.gca()
ax = style_axes(ax)

plt.tight_layout()

In [183]:
plt.savefig(r'/home/brandon/Documents/somitogenesis/paper_1/draft_1/figures/v2/interval_distribution_fig/amp_reg_interval_dists_v3.pdf')

In [ ]:
"""k_on regulation"""
p = Params()

# fixed params
p.translation_rate = translation_rate
p.mrna_decay_rate = mrna_decay_rate
p.protein_decay_rate = protein_decay_rate
p.Tmax = Tmax
p.dt = dt
p.max_time_to_next_reaction = max_time_to_next_reaction
p.n = hill_coefficient

# model specific params
p.transcription_rate_0 = 10.0
p.transcription_rate_1 = 0.0
p.KD = 1
p.n = 12
p.k_off0 = 2 * 0.08
p.k_on1 = 100 * 0.03 / p.KD ** p.n
p.k_off1 = 0.0
p.delay = 0

p_arr = [p] * n_replicates

with Pool(processes=15, initializer=init_pool_processes) as pool:
    res = pool.map(func, p_arr)

active_durations = [item for sublist in res for item in sublist[0]]
inactive_durations = [item for sublist in res for item in sublist[1]]

"""save the simulation output"""
output = [active_durations, inactive_durations]
output_file = r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/kon_reg.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(output, f)



In [184]:
"""plot kon reg"""

# load data
output_file = r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/kon_reg.pkl'
with open(output_file, 'rb') as f:
    active_durations, inactive_durations = pickle.load(f)

sim_active_dist_arr = bootstrap_simulated_distributions(active_durations, bins, n_bootstraps)
sim_inactive_dist_arr = bootstrap_simulated_distributions(inactive_durations, bins, n_bootstraps)

y_max = 0.2
yticks = [0, 0.1, 0.2]
plt.figure(figsize=(10, 5.5))
# plot active durations
plt.subplot(121)
counts, bins = np.histogram(active_durations, bins=bins)
prob_dens = counts / np.sum(counts) / np.diff(bins)
s = np.std(sim_active_dist_arr, axis=0)
plt.plot(bins[:-1], prob_dens, '-', linewidth=3, color=colors['green'])
plt.plot(bins[:-1], prob_dens, 'ko', markersize=16, markerfacecolor=colors['green'], alpha=0.75, label='data', linewidth=2)
plt.errorbar(bins[:-1], prob_dens, s, color='none',ecolor='k', linewidth=2, barsabove=True, capsize=4)


k = 1 / np.mean(active_durations)
x = np.linspace(0, 200, 1000)
y = k * np.exp(-k * x)
# plt.plot(x, y, 'k-', linewidth=4, label='stochastic bursts')

plt.ylim([0, y_max])
plt.xlim([-5, 65])
plt.xticks([0, 30, 60])
plt.yticks(yticks)
plt.xlabel('active interval (min)', fontsize=fontsize)
plt.ylabel('\n probability density (min)$^{-1}$', fontsize=fontsize)
ax = plt.gca()
ax = style_axes(ax)

plt.subplot(122)
counts, bins = np.histogram(inactive_durations, bins=bins)
prob_dens = counts / np.sum(counts) / np.diff(bins)
s = np.std(sim_inactive_dist_arr, axis=0)
plt.plot(bins[:-1], prob_dens, '-', linewidth=3, color=colors['green'])
plt.plot(bins[:-1], prob_dens, 'ko', markersize=16, markerfacecolor=colors['green'], alpha=0.75, label='data', linewidth=2)
plt.errorbar(bins[:-1], prob_dens, s, color='none',ecolor='k', linewidth=2, barsabove=True, capsize=4)


k = 1 / np.mean(inactive_durations)
x = np.linspace(0, 200, 1000)
y = k * np.exp(-k * x)
# plt.plot(x, y, 'k-', linewidth=4, label='stochastic bursts')

plt.ylim([0, y_max])
plt.xlim([-5, 65])
plt.xticks([0, 30, 60])
plt.yticks(yticks)
plt.xlabel('inactive interval (min)', fontsize=fontsize)
plt.ylabel('\n probability density (min)$^{-1}$', fontsize=fontsize)
ax = plt.gca()
ax = style_axes(ax)
      
plt.tight_layout()

In [185]:
plt.savefig(r'/home/brandon/Documents/somitogenesis/paper_1/draft_1/figures/v2/interval_distribution_fig/kon_reg_interval_dists_v3.pdf')

In [ ]:
"""k_off regulation"""
p = Params()

# fixed params
p.translation_rate = translation_rate
p.mrna_decay_rate = mrna_decay_rate
p.protein_decay_rate = protein_decay_rate
p.Tmax = Tmax
p.dt = dt
p.max_time_to_next_reaction = max_time_to_next_reaction
p.n = hill_coefficient

# model specific params
p.transcription_rate_0 = 10.0
p.transcription_rate_1 = 0.0
p.k_on0 = 0.08
p.k_off0 = 0.0
p.k_on1 = 0.0
p.k_off1 = 0.2
p.KD = 500.0
p.delay = 0

p_arr = [p] * n_replicates

with Pool(processes=15, initializer=init_pool_processes) as pool:
    res = pool.map(func, p_arr)

active_durations = [item for sublist in res for item in sublist[0]]
inactive_durations = [item for sublist in res for item in sublist[1]]

"""save the simulation output"""
output = [active_durations, inactive_durations]
output_file = r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/koff_reg.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(output, f)
    


In [186]:
"""plot duration reg"""

# load data
output_file = r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/koff_reg.pkl'
with open(output_file, 'rb') as f:
    active_durations, inactive_durations = pickle.load(f)

sim_active_dist_arr = bootstrap_simulated_distributions(active_durations, bins, n_bootstraps)
sim_inactive_dist_arr = bootstrap_simulated_distributions(inactive_durations, bins, n_bootstraps)

y_max = 0.2
yticks = [0, 0.10, 0.20]
plt.figure(figsize=(10, 5.5))
# plot active durations
plt.subplot(121)
counts, bins = np.histogram(active_durations, bins=bins)
prob_dens = counts / np.sum(counts) / np.diff(bins)
s = np.std(sim_active_dist_arr, axis=0)
plt.plot(bins[:-1], prob_dens, '-', linewidth=3, color=colors['green'])
plt.plot(bins[:-1], prob_dens, 'ko', markersize=16, markerfacecolor=colors['green'], alpha=0.75, label='data', linewidth=2)
plt.errorbar(bins[:-1], prob_dens, s, color='none',ecolor='k', linewidth=2, barsabove=True, capsize=4)


k = 1 / np.mean(active_durations)
x = np.linspace(0, 200, 1000)
y = k * np.exp(-k * x)
# plt.plot(x, y, 'k-', linewidth=4, label='stochastic bursts')

plt.ylim([0, y_max])
plt.xlim([-5, 65])
plt.xticks([0, 30, 60])
plt.yticks(yticks)
plt.xlabel('active interval (min)', fontsize=fontsize)
plt.ylabel('\n probability density (min)$^{-1}$', fontsize=fontsize)
ax = plt.gca()
ax = style_axes(ax)

plt.subplot(122)
counts, bins = np.histogram(inactive_durations, bins=bins)
prob_dens = counts / np.sum(counts) / np.diff(bins)
s = np.std(sim_inactive_dist_arr, axis=0)
plt.plot(bins[:-1], prob_dens, '-', linewidth=3, color=colors['green'])
plt.plot(bins[:-1], prob_dens, 'ko', markersize=16, markerfacecolor=colors['green'], alpha=0.75, label='data', linewidth=2)
plt.errorbar(bins[:-1], prob_dens, s, color='none',ecolor='k', linewidth=2, barsabove=True, capsize=4)

k = 1 / np.mean(inactive_durations)
x = np.linspace(0, 200, 1000)
y = k * np.exp(-k * x)
#plt.plot(x, y, 'k-', linewidth=4, label='stochastic bursts')

plt.ylim([0, y_max])
plt.xlim([-5, 65])
plt.xticks([0, 30, 60])
plt.yticks(yticks)
plt.xlabel('inactive interval (min)', fontsize=fontsize)
plt.ylabel('\n probability density (min)$^{-1}$', fontsize=fontsize)
ax = plt.gca()
ax = style_axes(ax)
      
plt.tight_layout()

In [187]:
plt.savefig(r'/home/brandon/Documents/somitogenesis/paper_1/draft_1/figures/v2/interval_distribution_fig/koff_reg_interval_dists_v3.pdf')

## Fig6F
Plot the experimental interval distributions and the null distributions from a model of unregulated bursts

In [112]:
"""load data for period and duration"""
# dorado
df = pd.read_pickle(path_to_dataset1)

with open(path_to_non_blank_timepoints,'rb') as fp:
    non_blank_timepoints = pickle.load(fp)


In [32]:
"""more functions for computing interval distributions. we also need func and init_pool_processes from the cell above."""
def compute_distributions(traces, bins):
    pulse_periods = []
    pulse_durations =[]
    pulse_quiets = []
    for i, trace in enumerate(traces):
        t_arr, inten_arr, nucleus = trace
        t_arr = non_blank_timepoints[t_arr.astype('int')]
        state = binarize_trace(inten_arr, t_arr, thresh=1.0, window_size=5)
        on_times, off_times = get_on_and_off_times(state, t_arr)
        if len(on_times) > 2:
            these_pulse_periods = np.diff(on_times)
            these_quiets = get_burst_inactive_durations(on_times, off_times)
            pulse_periods.extend([p for p in these_pulse_periods])
            these_pulse_durations = get_burst_durations(on_times, off_times)
            pulse_durations.extend([p for p in these_pulse_durations])
            pulse_quiets.extend([p for p in these_quiets])
            
    counts, bins = np.histogram(pulse_durations, bins=bins)
    prob_dens_durations = counts / np.sum(counts) / np.diff(bins)
    
    counts, bins = np.histogram(pulse_quiets, bins=bins)
    prob_dens_quiets = counts / np.sum(counts) / np.diff(bins)
            
    return prob_dens_durations, prob_dens_quiets


def bootstrap_distributions(traces, bins, n_bootstraps):
    duration_dist_arr = np.zeros((n_bootstraps, len(bins) - 1))
    quiet_dist_arr = np.zeros((n_bootstraps, len(bins) - 1))
    for i in range(int(n_bootstraps)):
        these_ids = np.random.randint(0, len(traces), len(traces), dtype='int')
        these_traces = [traces[j] for j in these_ids]
        
        duration_dist_arr[i], quiet_dist_arr[i] = compute_distributions(these_traces, bins)

    return duration_dist_arr, quiet_dist_arr 


def bootstrap_simulated_distributions(intervals, bins, n_bootstraps):
    interval_dist_arr = np.zeros((n_bootstraps, len(bins) - 1))
    for i in range(int(n_bootstraps)):
        these_ids = np.random.randint(0, len(intervals), len(intervals), dtype='int')
        these_intervals = [intervals[j] for j in these_ids]
        
        counts, bins = np.histogram(these_intervals, bins=bins)
        interval_dist_arr[i] = counts / np.sum(counts) / np.diff(bins)
        
    return interval_dist_arr 

In [114]:
"""extract lists of active and inactive intervals."""
traces = extract_traces(df, method='gauss3d_dog')
spots_per_trace = [np.sum(trace[1] > 0) for trace in traces]
traces = [traces[i] for i in range(len(traces)) if spots_per_trace[i] > 10]

# full dataset
pulse_periods = []
pulse_durations =[]
pulse_quiets = []
for i, trace in enumerate(traces):
    t_arr, inten_arr, nucleus = trace
    t_arr = non_blank_timepoints[t_arr.astype('int')]
    state = binarize_trace(inten_arr, t_arr, thresh=1.0, window_size=window_size)
    on_times, off_times = get_on_and_off_times(state, t_arr)
    if len(on_times) > 2:
        these_pulse_periods = np.diff(on_times)
        these_quiets = get_burst_inactive_durations(on_times, off_times)
        pulse_periods.extend([p for p in these_pulse_periods])
        these_pulse_durations = get_burst_durations(on_times, off_times)
        pulse_durations.extend([p for p in these_pulse_durations])
        pulse_quiets.extend([p for p in these_quiets])


        
# bootstrap 
n_bins = 30
data_bins = np.linspace(window_size, 60, n_bins)
n_bootstraps = 100
duration_dist_arr, quiet_dist_arr = bootstrap_distributions(traces, data_bins, n_bootstraps)
# plt.figure()
# plt.plot(bins[:-1], duration_dist_arr.T, linewidth=2)
                             
counts, data_bins = np.histogram(pulse_durations, bins=data_bins)
prob_dens_durations = counts / np.sum(counts) / np.diff(data_bins)

counts, data_bins = np.histogram(pulse_quiets, bins=data_bins)
prob_dens_quiets = counts / np.sum(counts) / np.diff(data_bins)



    



/home/brandon/Documents/Code/zebrafish-ms2-paper/zebrafish_ms2_paper/trace_analysis.py:41: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  inten_arr[i] = sub_sub_df.get(method)


In [30]:
"""simulation of unreglated bursting"""
p = Params()
p.transcription_rate_0 = 1.0
p.transcription_rate_1 = 0.0
p.translation_rate = 1.0
p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23

p.Tmax = 10_000
p.dt = 0.01
p.max_time_to_next_reaction = 1.0
p.k_on0 = 1 / np.mean(pulse_quiets)
p.k_off0 = 1 / np.mean(pulse_durations)
p.k_on1 = 0.0
p.k_off1 = 0.0
p.KD = 1.0
p.n = 2.0
p.delay = 0

p_arr = [p] * 20

with Pool(processes=15, initializer=init_pool_processes) as pool:
    res = pool.map(func, p_arr)

active_durations = [item for sublist in res for item in sublist[0]]
inactive_durations = [item for sublist in res for item in sublist[1]]

sim_active_dist_arr = bootstrap_simulated_distributions(active_durations, bins, n_bootstraps)
sim_inactive_dist_arr = bootstrap_simulated_distributions(inactive_durations, bins, n_bootstraps)



In [38]:
"""save the simulation output"""
output = [active_durations, inactive_durations]
output_file = r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/unregulated_bursting_data_compare.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(output, f)

In [158]:
# load data
output_file = r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/unregulated_bursting_data_compare.pkl'
with open(output_file, 'rb') as f:
    active_durations, inactive_durations = pickle.load(f)

sim_active_dist_arr = bootstrap_simulated_distributions(active_durations, bins, n_bootstraps)
sim_inactive_dist_arr = bootstrap_simulated_distributions(inactive_durations, bins, n_bootstraps)

plt.figure(figsize=(15, 6))
ymax = 0.1

"""plot durations"""
plt.subplot(121)

# analytic
k = 1 / np.mean(pulse_durations)
x = np.linspace(window_size, 200, 1000)
y = k * np.exp(-k * x)
#plt.plot(x, y, 'k-', linewidth=4, label='unregulated bursts \n (analytic)')

# sim
m = np.mean(sim_active_dist_arr, axis=0)
s = np.std(sim_active_dist_arr, axis=0)
l = m - s
u = m + s
plt.errorbar(bins[:-1], m, s, color=colors['purple'], linewidth=3, elinewidth=4, ecolor=colors['purple'], capsize=8, capthick=4)
plt.plot(bins[:-1], m, 'kd', markersize=18, markerfacecolor=colors['purple'], alpha=1, label='unregulated bursts \n (simulated)')
#plt.fill_between(bins[:-1], m, l, u, facecolor='m', alpha=0.75)
#plt.plot(bins[:-1], 'm-', linewidth=3, label='unregulated bursts \n (simulated)')

# data
m = np.mean(duration_dist_arr, axis=0)
s = np.std(duration_dist_arr, axis=0)
plt.errorbar(data_bins[:-1], m, s, color=colors['green'], linewidth=3, elinewidth=4, ecolor=colors['green'], capsize=8, capthick=4)
plt.plot(data_bins[:-1], m, 'ko', markersize=18, markerfacecolor=colors['green'], alpha=1, label='experimental data')

plt.xlabel('active interval (min)', fontsize=24)
plt.ylabel('\n probability density (min)$^{-1}$', fontsize=24)

plt.xlim([-5, 65])
plt.xticks([0, 30, 60])
plt.yticks([0, 0.05, 0.1, 0.15])
plt.ylim([0, ymax])
plt.legend(fontsize=18)

ax = plt.gca()
ax = style_axes(ax)
    
plt.tight_layout()


"""plot inactive durations"""
plt.subplot(122)

# analytic
k = 1 / np.mean(pulse_quiets)
x = np.linspace(window_size, 200, 1000)
y = k * np.exp(-k * x)
#plt.plot(x, y, 'k-', linewidth=4, label='unregulated bursts \n (analytic)')

# sim
m = np.mean(sim_inactive_dist_arr, axis=0)
s = np.std(sim_inactive_dist_arr, axis=0)
plt.errorbar(bins[:-1], m, s, color=colors['purple'], linewidth=3, elinewidth=4, ecolor=colors['purple'], capsize=8, capthick=4)
plt.plot(bins[:-1], m, 'kd', markersize=18, markerfacecolor=colors['purple'], alpha=1, label='unregulated bursts \n (simulated)')
#plt.fill_between(bins[:-1], m, l, u, facecolor='m', alpha=0.75)
#plt.plot(bins[:-1], 'm-', linewidth=3, label='unregulated bursts \n (simulated)')


# data
m = np.mean(quiet_dist_arr, axis=0)
s = np.std(quiet_dist_arr, axis=0)
plt.errorbar(data_bins[:-1], m, s, color=colors['green'], linewidth=3, elinewidth=4, ecolor=colors['green'], capsize=8, capthick=4)
plt.plot(data_bins[:-1], m, 'ko', markersize=18, markerfacecolor=colors['green'], alpha=1, label='experimental data')

plt.xlabel('inactive interval (min)', fontsize=24)
plt.ylabel('\n probability density (min)$^{-1}$', fontsize=24)

plt.xlim([-5, 65])
plt.xticks([0, 30, 60])
plt.yticks([0, 0.05, 0.1, 0.15])
plt.ylim([0, ymax])

#plt.legend(fontsize=18)

ax = plt.gca()
ax = style_axes(ax)

plt.tight_layout()
                         

In [159]:
plt.savefig(r'/home/brandon/Documents/somitogenesis/paper_1/draft_1/figures/v2/interval_distribution_fig/experimental_interval_dists_v2.pdf')